In [11]:
import pandas as pd
from rdkit import Chem
import os

folder_path = "PDBBind_processed"
folder_names = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]

folder_names.remove('1a50') # for some reason, this directory is empty

data = []
for folder_name in folder_names:
    mol_path = os.path.join(os.path.join(folder_path, folder_name), f'{folder_name}_ligand.mol2')
    try:
        mol = Chem.MolFromMol2File(mol_path)
    except:
        print(f"Error reading file: {mol_path}")
        continue
    
    if mol is not None:
        smiles = Chem.MolToSmiles(mol)
        data.append([folder_name, smiles])

df = pd.DataFrame(data, columns=['folder_name', 'smiles'])
df.to_csv('res.csv', index=False)




[14:33:58] 4ew3_ligand: warning - charged amidine and isFixed atom.
[14:33:58] Can't kekulize mol.  Unkekulized atoms: 0 26
[14:33:58] sanitise [14:33:58] 2q2n_ligand: [14:33:59] 2it4_ligand: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
[14:34:00] 6db4_ligand: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.


[14:34:01] Can't kekulize mol.  Unkekulized atoms: 3 4 19 20 22
[14:34:01] sanitise [14:34:01] 4kcx_ligand: [14:34:02] Can't kekulize mol.  Unkekulized atoms: 9 10 11 13 14 15 16 17 18
[14:34:02] sanitise [14:34:02] 1nu1_ligand: [14:34:03] WARNING: not removing hydrogen atom with neighbor that has non-tetrahedral stereochemistry
[14:34:03] WARNING: not removing hydrogen atom with neighbor that has non-tetrahedral stereochemistry
[14:34:03] WARNING: not removing hydrogen atom with neighbor that has non-tetrahedral stereochemistry
[14:34:03] WARNING: not removing hydrogen atom with neighbor that has non-tetrahedral stereochemistry
[14:34:03] Can't kekulize mol.  Unkekulized atoms: 9 10 27 30 32
[14:34:03] sanitise [14:34:03] 5wjj_ligand: [14:34:03] Can't kekulize mol.  Unkekulized atoms: 20 21 22 23 24
[14:34:03] sanitise [14:34:03] 5lwe_ligand: [14:34:04] Explicit valence for atom # 7 C, 5, is greater than permitted
[14:34:04] sanitise [14:34:04] 6pgx_ligand: [14:34:05] WARNING: not rem

In [16]:
import concurrent.futures
import requests
from tqdm import tqdm

# Define a function to download CIF file
def download_cif(folder_name, cif_url, cif_path):
    response = requests.get(cif_url)
    if response.status_code == 200:
        with open(cif_path, 'wb') as f:
            f.write(response.content)
    else:
        print(f"Error downloading CIF file for {folder_name}")

# Create a thread pool executor
executor = concurrent.futures.ThreadPoolExecutor()

# Iterate over each row in the DataFrame
for index, row in tqdm(df.iterrows(), total=len(df)):
    folder_name = row['folder_name']
    smiles = row['smiles']
    
    # Download the CIF file if it exists
    cif_url = f"https://files.rcsb.org/download/{folder_name}.cif"
    cif_path = f".tmp/{folder_name}.cif"
    if not os.path.exists(cif_path):
        # Submit the download task to the thread pool
        executor.submit(download_cif, folder_name, cif_url, cif_path)

# Shutdown the thread pool and wait for all tasks to complete
executor.shutdown(wait=True)


100%|██████████| 19097/19097 [00:03<00:00, 5300.53it/s]


In [2]:
import os

folder_path = "PDBBind_rcsb"
folder_names = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]

folders_with_more_than_two_files = 0
folders_with_two_files = 0

for folder_name in folder_names:
    folder_files = os.listdir(os.path.join(folder_path, folder_name))
    if len(folder_files) > 2:
        folders_with_more_than_two_files += 1
    elif len(folder_files) == 2:
        folders_with_two_files += 1

ratio = folders_with_more_than_two_files / (folders_with_two_files + folders_with_more_than_two_files)

ratio


0.21222685675154082

In [4]:
folders_with_two_files + folders_with_more_than_two_files

16063

In [3]:
len(folder_names)

16063

In [29]:
from rdkit import Chem
from rdkit.Chem import rdmolfiles


# 从 PDB 文件创建分子
mol = Chem.MolFromPDBFile('6ckl.pdb')

# 提取配体
ligand = [atom for atom in mol.GetAtoms() if atom.GetPDBResidueInfo().GetResidueName() == 'DAN']

# Save the ligand as an SDF file
rdmolfiles.MolToMolFile(ligand, 'ligand.sdf')


ArgumentError: Python argument types in
    rdkit.Chem.rdmolfiles.MolToMolFile(list, str)
did not match C++ signature:
    MolToMolFile(RDKit::ROMol mol, std::string filename, bool includeStereo=True, int confId=-1, bool kekulize=True, bool forceV3000=False)

In [24]:
import re
import requests

sequence_url = "https://www.rcsb.org/annotations/2uw8"
resp_seq = requests.get(sequence_url)
# The annotations website guarantees that there is only protein information
# print(resp_seq.text)

# FILEPATH: /home/t-kaiyuangao/Workspace/DynamicBind/PDBBindData/data_handle.ipynb
import re

pattern = re.compile(r'\[auth')
matches = pattern.finditer(resp_seq.text)

positions = [match.start() for match in matches]
# positions

positions == []

# if pattern:
#     # print(pattern.group(1)[1:])
# # if pattern:
#     print(pattern[0])

False

In [25]:
for _ in positions:
    print(resp_seq.text[_-2])

B
B
B
